In [1]:
import pandas as pd
from matplotlib import pyplot as plt

import matplotlib

matplotlib.use("agg")

from pathlib import Path


# Expects a pandas dataframe with the following columns:
#  - random (bool): the record was in the random sample run-in phase
#  - label (bool): the record was a include/exclude
#  - order (int): how many records were seen before this one
def assess_non_nice(df):
    if 'random' in df.columns:
        df = df[df['random'] == False]
    time_to_find = df[df['label'] == True]['order']
    # calculate lags between finds
    time_lags = time_to_find.diff()
    # the rolling average of the previous finds
    avg_preceding = time_lags.rolling(5, center=False).mean().shift()
    # count the proportion where the time lag drops significantly
    return sum(avg_preceding > time_lags * 4) / avg_preceding.count()


# Expects a pandas dataframe with the following columns:
#  - random (bool): the record was in the random sample run-in phase
#  - label (bool): the record was a include/exclude
def assess_gain(df):
    if 'random' in df.columns:
        df = df[df['random'] == False]
    # calculate the expected slope if screening randomly
    total_records = df['label'].count()
    total_found = df['label'].sum()
    slope = total_found / total_records
    # calculate the actual and expected found at each record screened
    n_found = df['label'].cumsum()
    exp_found = [i * slope for i in range(1, total_records + 1)]
    # sum proportion found at each record screened and normalize by number of records
    return sum((n_found - exp_found) / total_found) / total_records

gains = []
nns = []

for file in Path('../data/rankings').glob('*.feather'):
    df = pd.read_feather(file)
    df = df[df['random'] == False]
    df = df.filter(['order', 'label'])
    df['cumulative'] = df['label'].cumsum()

    nns.append(assess_non_nice(df))
    gains.append(assess_gain(df))

/tmp/ipykernel_96988/2172463228.py:24: RuntimeWarning: invalid value encountered in scalar divide
  return sum(avg_preceding > time_lags * 4) / avg_preceding.count()
/tmp/ipykernel_96988/2172463228.py:24: RuntimeWarning: invalid value encountered in scalar divide
  return sum(avg_preceding > time_lags * 4) / avg_preceding.count()
/tmp/ipykernel_96988/2172463228.py:24: RuntimeWarning: invalid value encountered in scalar divide
  return sum(avg_preceding > time_lags * 4) / avg_preceding.count()


In [2]:
df = pd.DataFrame({'gain': gains, 'stepping': nns})

In [3]:

df

,gain,stepping
0,0.326257,0.193548
1,0.417533,0.164179
2,-0.132743,NaN
3,0.335551,0.210526
4,-0.023897,0.196629
5,0.008299,0.166667
6,0.031133,0.193878
7,0.009221,0.250000
8,0.208391,0.222222
9,0.278040,0.330357


In [4]:
df['gain'].hist()
plt.savefig('../data/gains.png')

<Axes: >

In [7]:
plt.clf()
df['stepping'].hist()
plt.savefig('../data/steppings.png')